In [1]:
import yfinance as yf 
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.nn import Module ,Conv1d, Dropout,ReLU,Sequential,Linear
from torch.nn.utils import weight_norm
import numpy as np
from torch.nn import MSELoss,L1Loss
from torch.optim import Adam
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import copy

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LOOKBACK = 30
HORIZON = 5

In [3]:
gps_stock = pd.read_csv(r"C:\Users\User\Desktop\thesis_final\stock_price_and_news\stock_news\news_dataset\final_dataset\jpm_lstm_bert_new.csv")
print(gps_stock.shape)

(6736, 3)


In [4]:
gps_stock.drop(columns=['Date'], inplace = True)
gps_stock = gps_stock.values
print(gps_stock.shape)

(6736, 2)


In [5]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.9 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [6]:
def split_data(lookback,horizon, price):

    # (3000,2)

    data_initial = price
    window_size = lookback + horizon
    data = []
    for index in range(len(data_initial) - window_size):
        data.append(data_initial[index:index+ window_size])
    data = np.array(data) 
    # print(data.shape)
    # (300,6,2)

    # test_size = int(np.round(0.2*data.shape[0]))
    # train_set_size = data.shape[0] - (test_size)
    train_set = data
    
    x_train = train_set[:,:lookback,:]
    # print(x_train.shape)
    

    y_train = train_set[:,lookback:lookback + horizon, 1]
    y_train = y_train.reshape(-1,horizon)
    # print(y_train.shape)

    # test_set = data[train_set_size:]
    # x_test = test_set[:,:lookback]
    # y_test = test_set[:,lookback:lookback+ horizon]
    # y_test = y_test.reshape(-1,horizon)
    
    #x_train = x_train.reshape(x_train.shape[0],x_train.shape[2],x_train.shape[1])
    #y_train = y_train.reshape(y_train.shape[0],y_train.shape[2],y_train.shape[1])
    # x_test = x_test.reshape(x_test.shape[0],x_test.shape[2],x_test.shape[1])
    #y_test = y_test.reshape(y_test.shape[0],y_test.shape[2],y_test.shape[1])


    return [x_train, y_train]

In [7]:
class cnn_dataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y 
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [8]:
device = 'cpu'

In [9]:



class PrintLayer(Module):
    def forward(self,X):
        print(X.shape)
        return X
class slider1d(Module):
    def __init__(self,slider_size):
        super(slider1d,self).__init__()
        self.slider_size = slider_size
    def forward(self,x):
        return x[:,:,:-self.slider_size].contiguous()
class temporalblock(Module):
    def __init__(self,n_inputs, n_outputs,kernel_size, stride,dilation, padding,dropout = 0.2):
        super(temporalblock,self).__init__()

        # print(f"{n_inputs=},{n_outputs=}")
        self.cnn1 = weight_norm(Conv1d(n_inputs, n_outputs,kernel_size,stride = stride, padding= padding, dilation = dilation))
        self.slider1 = slider1d(padding)
        self.relu1 = ReLU()
        self.dropout1 = Dropout(dropout)

        self.cnn2 = weight_norm(Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.slider2 = slider1d(padding)
        self.relu2 = ReLU()
        self.dropout2 = Dropout(dropout)
        self.tcn = Sequential(self.cnn1, self.slider1, self.relu1, self.dropout1,
                                 self.cnn2, self.slider2, self.relu2, self.dropout2)
        self.downsample = Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = ReLU()
        self.init_weights()
    def init_weights(self):
        self.cnn1.weight.data.normal_(0, 0.01)
        self.cnn2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)
    def forward(self, x):
        out = self.tcn(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)
class TemporalCovNet(Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalCovNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [temporalblock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        # 构建tcn网络
        self.tcn = TemporalCovNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        # 将网络输出映射到实际输出，使用线性映射
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])




           

In [10]:
train_losses = []
valid_losses = []

In [11]:
def Train(data_loader, model,optimizer, criterion,) :
    running_loss = 0
    
    for (X,y) in (data_loader):
        X = X.to(device)
        y = y.to(device)
       
        #l2_lambda = 0.001
        #l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        #loss = loss + l2_lambda *l2_norm
        optimizer.zero_grad()
        preds = model(X.float())
        loss = criterion(preds, y.float())
       
        
        loss.backward()
        optimizer.step()
        running_loss += np.sqrt(loss.item())

    train_loss = running_loss/len(data_loader)
    # print(f'train_loss{train_loss}')

    return train_loss, model

    



def Valid(data_loader,  model, optimizer, criterion):
    """
    return y_true, y_hat
    """
    running_loss = 0
    
    
    y_hat = []
    y_true = []
    with torch.no_grad():
        for (X, y) in (data_loader):
            X = X.to(device)
            y = y.to(device)
         
            y_true = [*y_true,*(y.reshape(-1).tolist()) ]
            optimizer.zero_grad()
            preds = model(X.float())
         
            y_hat = [*y_hat, *(preds.reshape(-1).tolist())]
      
            loss = criterion(preds,y.float())
            running_loss += np.sqrt(loss.item())
            
        valid_loss = running_loss/len(data_loader)
        
        # print(f'valid_loss {valid_loss}')

    return y_true, y_hat, valid_loss, model


In [12]:
spliter = BlockingTimeSeriesSplit(n_splits = 10)

cv_test_losses = []
cv_train_losses = []
cv_val_losses = []

for trainval_idx ,test_idx in spliter.split(gps_stock):
    # print(trainval_idx)
    # print(test_idx)

    train_idx = trainval_idx[ : int(len(trainval_idx)*0.8)]
    val_idx   = trainval_idx[int(len(trainval_idx)*0.8) : ]

    x_train_unsplit = gps_stock[train_idx,:]
    x_val_unsplit  = gps_stock[val_idx,:]
    x_test_unsplit = gps_stock[test_idx,:]

    # print(x_train_unsplit.shape)
    # adadasdas
    # print(train_idx)
    # print(val_idx)

    x_train_gps, y_train_gps = split_data(LOOKBACK, HORIZON, x_train_unsplit)
    x_val_gps, y_val_gps = split_data(LOOKBACK, HORIZON, x_val_unsplit)
    x_test_gps, y_test_gps = split_data(LOOKBACK, HORIZON, x_test_unsplit)
    input_channels = x_train_gps.shape[1]
    n_classes = y_train_gps.shape[1]
    channel_sizes = [LOOKBACK]*8
    # print(x_train_gps.shape)
    # print(y_train_gps.shape)

    # print(x_val_gps.shape)
    # print(y_val_gps.shape)

    # print(x_test_gps.shape)
    # print(y_test_gps.shape)


    train_dataset_gps = cnn_dataset(x_train_gps,y_train_gps)
    train_loader_gps = torch.utils.data.DataLoader(train_dataset_gps, batch_size = 16, shuffle = False)
    test_dataset_gps = cnn_dataset(x_test_gps,y_test_gps)
    test_loader_gps = torch.utils.data.DataLoader(test_dataset_gps,batch_size=16,shuffle=False)
    valid_dataset_gps = cnn_dataset(x_val_gps,y_val_gps)
    valid_loader_gps = torch.utils.data.DataLoader(valid_dataset_gps,batch_size=16,shuffle=False)
    # print(len(train_dataset_gps))
    model = TCN(input_channels, n_classes, channel_sizes, kernel_size=5, dropout=0.2)
    model = model.to(device)
    criterion = MSELoss(reduction= 'mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4, weight_decay= 1e-10)

    train_losses = []
    val_losses = []

    epochs = 200

    best_val_loss = 1e+9

    for epoch in range(epochs):
        # print(epoch)
        # print('epochs {}/{}'.format(epoch+1,epochs))
        train_loss , model               = Train(train_loader_gps, model, optimizer, criterion)
        y_true, y_hat, valid_loss , model = Valid(valid_loader_gps, model, optimizer, criterion)

        # print(train_loss)
        train_losses.append(train_loss)
        val_losses.append(valid_loss)

        curr_val_loss = valid_loss

        if curr_val_loss < best_val_loss:
            best_model = copy.deepcopy(model)
            best_val_loss = curr_val_loss
            print("Best model : ", epoch)

    
    y_test_true, y_test_hat, test_loss , _ = Valid(test_loader_gps, best_model, optimizer, criterion)

    print(test_loss)
    cv_test_losses.append(test_loss)
    cv_train_losses.append(train_losses)
    cv_val_losses.append(val_losses)
    # break

print(cv_test_losses)
print(sum(cv_test_losses) / len(cv_test_losses))

Best model :  0
Best model :  1
Best model :  3
Best model :  4
Best model :  5
Best model :  20
Best model :  21
Best model :  35
Best model :  36
Best model :  45
Best model :  56
Best model :  64
Best model :  65
Best model :  71
Best model :  77
Best model :  90
Best model :  97
Best model :  105
Best model :  123
Best model :  137
Best model :  156
Best model :  176
Best model :  185
0.005513350113281007
Best model :  0
Best model :  1
Best model :  32
Best model :  33
Best model :  34
Best model :  35
Best model :  36
Best model :  37
Best model :  38
Best model :  39
Best model :  40
Best model :  41
Best model :  42
Best model :  43
Best model :  44
Best model :  45
Best model :  47
Best model :  48
Best model :  51
Best model :  52
Best model :  54
Best model :  58
Best model :  68
Best model :  72
Best model :  73
Best model :  74
Best model :  75
Best model :  77
Best model :  78
Best model :  88
Best model :  93
Best model :  94
Best model :  99
Best model :  101
Best model